In [9]:
from ngsolve import *
import numpy as np
import scipy.optimize
import datetime
import sys

from netgen.csg import *
from netgen.meshing import MeshingParameters, meshsize
from ngsolve.webgui import Draw
import matplotlib.pyplot as plt
import scipy.optimize

class gel_3D:
    def __init__(self, folder_name_suffix, length=90.0, thickness=3.0, width =23.5, delta=0.9):
        self.phi0 = 0.2
        self.entropic_unit = 136.6  # measured in MPa
        self.G = 0.13               # measured in MPa
        self.gamma = self.G/self.entropic_unit # 0.0009516837481698391 self.compute_gamma( lamb =1.4874):
        self.lambda_target = 1.99  # 0.13 = Gfun(1.99)
        self.chi =  0.348 # compute_chi(phi0=0.2035, gamma =0.0009516837481698391, J=J_iso)
        self.density =  1.23 # measured in [g/mL]

        self.L = length      # measured in mm
        self.d = thickness    # measured in mm
        self.w = width # measured in mm
        self.delta = delta   # dimensionless
        
        self.filename_suffix = "_d={:.2f}_delta={:.3f}".format(self.d, self.delta)
        self.folder_name_suffix = folder_name_suffix; 

        self.geometry_bonded()

    def geometry_bonded(self):
        left  = Plane (Pnt(-0.5*self.L,0,0), Vec(-1,0,0) )
        right = Plane (Pnt(0.5*self.L,0,0), Vec( 1,0,0) )
        bot   = Plane (Pnt(0,0,0), Vec(0,-1,0) ).bc('bottom')
        top   = Plane (Pnt(0,self.d,0), Vec(0,1,0) )
        back  = Plane (Pnt(0,0,-0.5*self.w), Vec(0,0, -1) )
        front = Plane (Pnt(0,0,0.5*self.w), Vec(0,0,1) )

        brick = left * right * front * back * bot * top

        self.geo = CSGeometry()
        self.geo.Add (brick)

### MAIN ###

L = 90.0; d=3.00; w=23.5; delta=1.0;
maxh = 0.65
order = 2

folder_name_suffix = str(int(d)) + '_' + str(int(d%1*100)).zfill(2)

gel = gel_3D(folder_name_suffix = folder_name_suffix, \
    length=L, thickness=d,width=w, delta=delta)

mesh = Mesh(gel.geo.GenerateMesh(maxh=maxh))

mesh_filename = 'meshes' + folder_name_suffix + '/mesh9'
mesh.ngmesh.Save(mesh_filename)

Draw(mesh)

fes = VectorH1(mesh, order=2, dirichlet="bottom")
print('nDoF = {}'.format(fes.ndof))


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

nDoF = 508089
